# Using data collators with 🤗 Transformers and Datasets
> Calculating top losses by example

- toc: true 
- badges: false
- comments: false
- categories: [til,huggingface,transformers,nlp,datasets]
- image: images/icml.png

In [1]:
#hide
import warnings
import datasets
import transformers

warnings.filterwarnings("ignore")
datasets.logging.set_verbosity_error()
transformers.logging.set_verbosity_error()

Recently, [Sylvain Gugger](https://twitter.com/GuggerSylvain?s=20) from HuggingFace has created some nice tutorials on using `transformers` for [text classification](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb) and [named entity recognition](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=545PP3o8IrJV). One trick that caught my attention was the use of a _data collator_ in the trainer, which automatically pads the model inputs in a batch to the length of the longest example. This bypasses the need to set a _global_ maximum sequence length, and in practice leads to faster training since we perform fewer redundant computations on the padded tokens.

I wanted to use to use a data collator for both training _and_ error analysis (e.g. by inspecting the top losses of the model). One problem: each batch in a `Dataset.map` function is a dictionary of lists, while the data collators in `transformers` expect a list of dictionaries.

```python
# get from Trainer.data_collator
data_collator = ...

def processing_function(batch):
    # convert dict of lists to list of dicts
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    # pad inputs and (possibly) labels
    batch = data_collator(features)
    ...
    return batch
```

In [1]:
from transformers import DataCollatorWithPadding

In [4]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
batch = {'input_ids':[1,2,3], 'attention_mask':[1,]}

In [2]:
from datasets import load_dataset

imdb = load_dataset('imdb', split='train').train_test_split(train_size=750, test_size=250)
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

In [3]:
#hide_output
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

num_labels = 2
model_name = 'distilbert-base-cased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

In [4]:
def tokenize(batch): return tokenizer(batch['text'], truncation=True)

imdb_enc = imdb.map(tokenize, batched=True)
imdb_enc

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 750
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 250
    })
})

In [5]:
imdb_enc['test'][:1].keys()

odict_keys(['attention_mask', 'input_ids', 'label', 'text'])

In [6]:
import numpy as np
from datasets import load_metric

accuracy_score = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_score.compute(predictions=predictions, references=labels)

In [7]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(imdb_enc['train']) // batch_size

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
)

In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=imdb_enc['train'],
    eval_dataset=imdb_enc['test'],
    tokenizer=tokenizer
)

trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy
1,0.657553,0.601859,0.812000


In [9]:
data_collator = trainer.data_collator

In [10]:
def forward_pass_with_label(batch):
    # Convert dict of lists to list of dicts
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    # Pad inputs and labels
    batch = data_collator(features)
    input_ids = torch.tensor(batch["input_ids"], device=device)
    attention_mask = torch.tensor(batch["attention_mask"], device=device)
    labels = torch.tensor(batch["labels"], device=device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        batch["predicted_label"] = torch.argmax(output.logits, axis=1)

    loss = torch.nn.functional.cross_entropy(output.logits, labels, reduction="none")

    batch["loss"] = loss
    
    # Datasets requires list of NumPy array data types
    for k, v in batch.items():
        batch[k] = v.cpu().numpy()

    return batch

In [11]:
losses_ds = imdb_enc['test'].map(forward_pass_with_label, batched=True, batch_size=16, remove_columns=['input_ids'])
losses_ds.set_format('pandas')
losses_df = losses_ds[:][['label', 'predicted_label', 'loss']]

In [12]:
losses_df['text'] = imdb['test']['text']

In [13]:
losses_df.head()

,label,predicted_label,loss,text
0,1,0,0.698180,"While the original 1932 version, with Preston ..."
1,1,1,0.459517,Ironically the most talked-about American film...
2,0,0,0.602936,"Bingo is the game, bullshit is the name. Rarel..."
3,1,1,0.475849,There are so many reasons as to why I rate the...
4,0,0,0.606920,I thought watching employment videos on corpor...


In [15]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
losses_df.sort_values("loss", ascending=False).head()

label  predicted_label      loss  \
199      0                1  1.011133   
129      1                0  0.966005   
102      0                1  0.928655   
65       0                1  0.921415   
131      0                1  0.901468   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              